In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, chi2
import random
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
# import shap
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

In [ ]:
data = pd.read_csv('df_data (1).csv', sep=";")
data.head(5)

<ipython-input-227-c7226d78dde0>:1: DtypeWarning: Columns (11,14,20,23,31,33,41,46,49) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('df_data (1).csv', sep=";")


,date,time,Жажда,кофеин,startAnswering,Зевания,Тревога,ГБ новая,Алкоголь,Депрессия,...,заметил провокатор,усиление движением,Продолжительность сна,чувствительность к звуку,чувствительность к свету,чувствительность к запахам,ГБ продолжение,Окончание боли,комментарий,какой триггер
0,2023-01-25,15:00,да,да,1674640902,нет,1,нет,нет,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-25,21:00,нет,нет,1674662469,нет,1,нет,нет,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-26,10:00,да,нет,1674710845,нет,1,да,нет,2,...,нет,да,06:30,да,нет,нет,NaN,NaN,NaN,NaN
3,2023-01-26,15:00,да,нет,1674727248,нет,1,NaN,нет,1,...,NaN,NaN,NaN,NaN,NaN,NaN,да,10:00,NaN,NaN
4,2023-01-26,21:00,нет,нет,1674749597,нет,1,нет,нет,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data["ГБ новая"] = data["ГБ новая"].replace({'да': 1, 'нет': 0})

In [ ]:
data["ГБ новая"]

0        0.0
1        0.0
2        1.0
3        NaN
4        0.0
        ... 
34826    1.0
34827    NaN
34828    1.0
34829    1.0
34830    NaN
Name: ГБ новая, Length: 34831, dtype: float64

In [ ]:
data_not_import = data[["startAnswering", "id", "комментарий"]]
data = data.drop(columns=["startAnswering", "id", "комментарий"], axis=1)

In [ ]:
categorical_columns = ["Название", "какой день", "какой триггер", "односторонняя",
                       "сыр, шоко, цитрус", "Физическая ативность", "Продолжительность сна", "Начало боли", "Окончание боли"]
binary_columns = ['аура', 'Жажда', 'кофеин', 'Зевания', 'тошнота', 'ГБ новая', 'Алкоголь',
                  "Перелеты", "боль в шее","пульсация", "вегетатика", "подташнивает",
                  "Обезболивающее", "мочеиспускание", "Напряжение глаз", "Воды достаточно?", "1 день менструации", "Были резкие запахи?",
                  "заметил провокатор", "усиление движением", "Пропуск приема пищи", "Чувствит кожи головы", "Сложность концентрации",
                  "Больше света, чем обычно", "чувствительность к звуку", "чувствительность к свету", "чувствительность к запахам",
                  "ГБ продолжение", "Хотелось шоколада", "Больше звука чем обычно"]
quantitative_columns = ["Тревога", "ВАШ макс", "Депрессия", "Сонливость", "Чувство голода", "Ожидаете мигрень", "Работосособность",
                     "Чувство усталости", "Раздражительность", "Качество сна", "Свежесть после сна", "% заполнения дневника",
                        "Чувствительность к звуку", "Чувствительность к свету", "Чувствительность к запахам"
                        ]
time_data =['time']
date_data = ["date"]

In [ ]:
longest_string_index = data[['time', "Начало боли", "Продолжительность сна", "Окончание боли"]].apply(lambda x: ''.join(x.astype(str)), axis=1).str.len().idxmax()
longest_string = data.loc[longest_string_index, ['time', "Начало боли", "Продолжительность сна", "Окончание боли"]]
longest_string

time                                                                10:00
Начало боли                                                           NaN
Продолжительность сна    Сложно оценить, т.к просыпалась каждые 30-40 мин
Окончание боли                                                        NaN
Name: 33545, dtype: object

In [ ]:
data = data.drop(columns=["Боль сейчас"], axis=1)
data = data.drop(data.iloc[:, 27:58],axis = 1)
data = data.drop(columns=["Ожидаете мигрень"], axis=1)

In [ ]:
categorical_columns = [
                       "сыр, шоко, цитрус", "Физическая ативность", ]
binary_columns = ['Жажда', 'кофеин', 'Зевания', 'ГБ новая', 'Алкоголь'
                  , "боль в шее", "вегетатика", "подташнивает",
                   "мочеиспускание", "Напряжение глаз", "Воды достаточно?", "Были резкие запахи?",
                    "Пропуск приема пищи", "Чувствит кожи головы",
                   "Хотелось шоколада"]
quantitative_columns = ["Тревога", "Депрессия", "Сонливость", "Чувство голода",
                        "Работосособность", "Чувство усталости", "Раздражительность"

                        ]
time_data =['time']
date_data = ["date"]

In [ ]:
mode_values = data.mode().iloc[0]

# Fill NaN values with mode values in all columns
data = data.fillna(mode_values)
data

,date,time,Жажда,кофеин,Зевания,Тревога,ГБ новая,Алкоголь,Депрессия,боль в шее,...,Воды достаточно?,"сыр, шоко, цитрус",Работосособность,Хотелось шоколада,Чувство усталости,Раздражительность,Были резкие запахи?,Пропуск приема пищи,Чувствит кожи головы,Физическая ативность
0,2023-01-25,15:00,да,да,нет,1,0.0,нет,1,да,...,да,нет,1,да,2,1,нет,нет,нет,совсем нет
1,2023-01-25,21:00,нет,нет,нет,1,0.0,нет,1,нет,...,да,нет,3,нет,3,1,нет,нет,нет,слабо
2,2023-01-26,10:00,да,нет,нет,1,1.0,нет,2,да,...,да,нет,2,нет,2,2,да,нет,да,совсем нет
3,2023-01-26,15:00,да,нет,нет,1,0.0,нет,1,да,...,да,нет,1,да,2,1,нет,нет,да,совсем нет
4,2023-01-26,21:00,нет,нет,нет,1,0.0,нет,1,нет,...,да,нет,3,нет,3,1,нет,нет,нет,совсем нет
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34826,2021-03-17,15:00,да,да,нет,3,1.0,да,3,нет,...,да,да,3,да,5,3,нет,да,нет,совсем нет
34827,2021-08-16,10:00,нет,да,нет,2,0.0,нет,3,да,...,да,да,3,да,4,4,нет,да,нет,слабо
34828,2021-08-16,21:00,да,да,да,3,1.0,нет,3,да,...,да,да,4,да,5,2,нет,нет,да,слабо
34829,2021-08-18,15:00,да,да,нет,3,1.0,нет,3,да,...,да,да,3,нет,4,4,нет,нет,нет,слабо


In [ ]:
data["ГБ новая"] = data["ГБ новая"].astype(int)

In [ ]:
data

,date,time,Жажда,кофеин,Зевания,Тревога,ГБ новая,Алкоголь,Депрессия,боль в шее,...,Воды достаточно?,"сыр, шоко, цитрус",Работосособность,Хотелось шоколада,Чувство усталости,Раздражительность,Были резкие запахи?,Пропуск приема пищи,Чувствит кожи головы,Физическая ативность
0,2023-01-25,15:00,да,да,нет,1,0,нет,1,да,...,да,нет,1,да,2,1,нет,нет,нет,совсем нет
1,2023-01-25,21:00,нет,нет,нет,1,0,нет,1,нет,...,да,нет,3,нет,3,1,нет,нет,нет,слабо
2,2023-01-26,10:00,да,нет,нет,1,1,нет,2,да,...,да,нет,2,нет,2,2,да,нет,да,совсем нет
3,2023-01-26,15:00,да,нет,нет,1,0,нет,1,да,...,да,нет,1,да,2,1,нет,нет,да,совсем нет
4,2023-01-26,21:00,нет,нет,нет,1,0,нет,1,нет,...,да,нет,3,нет,3,1,нет,нет,нет,совсем нет
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34826,2021-03-17,15:00,да,да,нет,3,1,да,3,нет,...,да,да,3,да,5,3,нет,да,нет,совсем нет
34827,2021-08-16,10:00,нет,да,нет,2,0,нет,3,да,...,да,да,3,да,4,4,нет,да,нет,слабо
34828,2021-08-16,21:00,да,да,да,3,1,нет,3,да,...,да,да,4,да,5,2,нет,нет,да,слабо
34829,2021-08-18,15:00,да,да,нет,3,1,нет,3,да,...,да,да,3,нет,4,4,нет,нет,нет,слабо


In [ ]:
target = data["ГБ новая"]

In [ ]:
y = data["ГБ новая"].values

In [ ]:
data = data.drop(columns = ["ГБ новая"], axis=1)

In [ ]:
binary_columns = ['Жажда', 'кофеин', 'Зевания',  'Алкоголь'
                  , "боль в шее", "вегетатика", "подташнивает",
                   "мочеиспускание", "Напряжение глаз", "Воды достаточно?", "Были резкие запахи?",
                    "Пропуск приема пищи", "Чувствит кожи головы",
                   "Хотелось шоколада"]

In [ ]:
for column in binary_columns:
    data[column] = data[column].replace({'да': 1, 'нет': 0})

In [ ]:
for column in time_data:
    data[column] = pd.to_datetime(data[column], format='%H:%M')

for column in time_data:
    time_components = [f'{column}_hour', f'{column}_minute']

    data[time_components] = data[column].apply(lambda x: pd.Series([x.hour, x.minute]))
    data_encoded = pd.get_dummies(data, columns=time_components)

In [ ]:
data

,date,time,Жажда,кофеин,Зевания,Тревога,Алкоголь,Депрессия,боль в шее,Сонливость,...,Работосособность,Хотелось шоколада,Чувство усталости,Раздражительность,Были резкие запахи?,Пропуск приема пищи,Чувствит кожи головы,Физическая ативность,time_hour,time_minute
0,2023-01-25,1900-01-01 15:00:00,1,1,0,1,0,1,1,3,...,1,1,2,1,0,0,0,совсем нет,15,0
1,2023-01-25,1900-01-01 21:00:00,0,0,0,1,0,1,0,3,...,3,0,3,1,0,0,0,слабо,21,0
2,2023-01-26,1900-01-01 10:00:00,1,0,0,1,0,2,1,2,...,2,0,2,2,1,0,1,совсем нет,10,0
3,2023-01-26,1900-01-01 15:00:00,1,0,0,1,0,1,1,2,...,1,1,2,1,0,0,1,совсем нет,15,0
4,2023-01-26,1900-01-01 21:00:00,0,0,0,1,0,1,0,3,...,3,0,3,1,0,0,0,совсем нет,21,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34826,2021-03-17,1900-01-01 15:00:00,1,1,0,3,1,3,0,2,...,3,1,5,3,0,1,0,совсем нет,15,0
34827,2021-08-16,1900-01-01 10:00:00,0,1,0,2,0,3,1,4,...,3,1,4,4,0,1,0,слабо,10,0
34828,2021-08-16,1900-01-01 21:00:00,1,1,1,3,0,3,1,4,...,4,1,5,2,0,0,1,слабо,21,0
34829,2021-08-18,1900-01-01 15:00:00,1,1,0,3,0,3,1,5,...,3,0,4,4,0,0,0,слабо,15,0


In [ ]:
data = pd.get_dummies(data, columns=categorical_columns+quantitative_columns+binary_columns)

In [ ]:
data

,date,time,time_hour,time_minute,"сыр, шоко, цитрус_да","сыр, шоко, цитрус_нет",Физическая ативность_Слабо,Физическая ативность_да,Физическая ативность_интенсивная,Физическая ативность_интенсивно,...,Воды достаточно?_0,Воды достаточно?_1,Были резкие запахи?_0,Были резкие запахи?_1,Пропуск приема пищи_0,Пропуск приема пищи_1,Чувствит кожи головы_0,Чувствит кожи головы_1,Хотелось шоколада_0,Хотелось шоколада_1
0,2023-01-25,1900-01-01 15:00:00,15,0,0,1,0,0,0,0,...,0,1,1,0,1,0,1,0,0,1
1,2023-01-25,1900-01-01 21:00:00,21,0,0,1,0,0,0,0,...,0,1,1,0,1,0,1,0,1,0
2,2023-01-26,1900-01-01 10:00:00,10,0,0,1,0,0,0,0,...,0,1,0,1,1,0,0,1,1,0
3,2023-01-26,1900-01-01 15:00:00,15,0,0,1,0,0,0,0,...,0,1,1,0,1,0,0,1,0,1
4,2023-01-26,1900-01-01 21:00:00,21,0,0,1,0,0,0,0,...,0,1,1,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34826,2021-03-17,1900-01-01 15:00:00,15,0,1,0,0,0,0,0,...,0,1,1,0,0,1,1,0,0,1
34827,2021-08-16,1900-01-01 10:00:00,10,0,1,0,0,0,0,0,...,0,1,1,0,0,1,1,0,0,1
34828,2021-08-16,1900-01-01 21:00:00,21,0,1,0,0,0,0,0,...,0,1,1,0,1,0,0,1,0,1
34829,2021-08-18,1900-01-01 15:00:00,15,0,1,0,0,0,0,0,...,0,1,1,0,1,0,1,0,1,0


In [ ]:
data.iloc[:, 0:2]

,date,time
0,2023-01-25,1900-01-01 15:00:00
1,2023-01-25,1900-01-01 21:00:00
2,2023-01-26,1900-01-01 10:00:00
3,2023-01-26,1900-01-01 15:00:00
4,2023-01-26,1900-01-01 21:00:00
...,...,...
34826,2021-03-17,1900-01-01 15:00:00
34827,2021-08-16,1900-01-01 10:00:00
34828,2021-08-16,1900-01-01 21:00:00
34829,2021-08-18,1900-01-01 15:00:00


In [ ]:
import pandas as pd

# Assuming your data is stored in a DataFrame called "df"
data["time"] = pd.to_datetime(data["time"]).dt.time


In [ ]:
data

,date,time,time_hour,time_minute,"сыр, шоко, цитрус_да","сыр, шоко, цитрус_нет",Физическая ативность_Слабо,Физическая ативность_да,Физическая ативность_интенсивная,Физическая ативность_интенсивно,...,Воды достаточно?_0,Воды достаточно?_1,Были резкие запахи?_0,Были резкие запахи?_1,Пропуск приема пищи_0,Пропуск приема пищи_1,Чувствит кожи головы_0,Чувствит кожи головы_1,Хотелось шоколада_0,Хотелось шоколада_1
0,2023-01-25,15:00:00,15,0,0,1,0,0,0,0,...,0,1,1,0,1,0,1,0,0,1
1,2023-01-25,21:00:00,21,0,0,1,0,0,0,0,...,0,1,1,0,1,0,1,0,1,0
2,2023-01-26,10:00:00,10,0,0,1,0,0,0,0,...,0,1,0,1,1,0,0,1,1,0
3,2023-01-26,15:00:00,15,0,0,1,0,0,0,0,...,0,1,1,0,1,0,0,1,0,1
4,2023-01-26,21:00:00,21,0,0,1,0,0,0,0,...,0,1,1,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34826,2021-03-17,15:00:00,15,0,1,0,0,0,0,0,...,0,1,1,0,0,1,1,0,0,1
34827,2021-08-16,10:00:00,10,0,1,0,0,0,0,0,...,0,1,1,0,0,1,1,0,0,1
34828,2021-08-16,21:00:00,21,0,1,0,0,0,0,0,...,0,1,1,0,1,0,0,1,0,1
34829,2021-08-18,15:00:00,15,0,1,0,0,0,0,0,...,0,1,1,0,1,0,1,0,1,0


In [ ]:
data.columns

Index(['date', 'time', 'time_hour', 'time_minute', 'сыр, шоко, цитрус_да',
       'сыр, шоко, цитрус_нет', 'Физическая ативность_Слабо',
       'Физическая ативность_да', 'Физическая ативность_интенсивная',
       'Физическая ативность_интенсивно',
       ...
       'Воды достаточно?_0', 'Воды достаточно?_1', 'Были резкие запахи?_0',
       'Были резкие запахи?_1', 'Пропуск приема пищи_0',
       'Пропуск приема пищи_1', 'Чувствит кожи головы_0',
       'Чувствит кожи головы_1', 'Хотелось шоколада_0', 'Хотелось шоколада_1'],
      dtype='object', length=113)

In [ ]:

# Convert "date" and "time" columns to a single datetime column
data["datetime"] = pd.to_datetime(data["date"] + " " + data["time"].astype(str))

# Set the datetime column as the index
data.set_index("datetime", inplace=True)

# Drop unnecessary columns
data.drop(["date", "time"], axis=1, inplace=True)


In [ ]:
data

,time_hour,time_minute,"сыр, шоко, цитрус_да","сыр, шоко, цитрус_нет",Физическая ативность_Слабо,Физическая ативность_да,Физическая ативность_интенсивная,Физическая ативность_интенсивно,Физическая ативность_нет,Физическая ативность_слабо,...,Воды достаточно?_0,Воды достаточно?_1,Были резкие запахи?_0,Были резкие запахи?_1,Пропуск приема пищи_0,Пропуск приема пищи_1,Чувствит кожи головы_0,Чувствит кожи головы_1,Хотелось шоколада_0,Хотелось шоколада_1
datetime,,,,,,,,,,,,,,,,,,,,,
2023-01-25 15:00:00,15,0,0,1,0,0,0,0,0,0,...,0,1,1,0,1,0,1,0,0,1
2023-01-25 21:00:00,21,0,0,1,0,0,0,0,0,1,...,0,1,1,0,1,0,1,0,1,0
2023-01-26 10:00:00,10,0,0,1,0,0,0,0,0,0,...,0,1,0,1,1,0,0,1,1,0
2023-01-26 15:00:00,15,0,0,1,0,0,0,0,0,0,...,0,1,1,0,1,0,0,1,0,1
2023-01-26 21:00:00,21,0,0,1,0,0,0,0,0,0,...,0,1,1,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-17 15:00:00,15,0,1,0,0,0,0,0,0,0,...,0,1,1,0,0,1,1,0,0,1
2021-08-16 10:00:00,10,0,1,0,0,0,0,0,0,1,...,0,1,1,0,0,1,1,0,0,1
2021-08-16 21:00:00,21,0,1,0,0,0,0,0,0,1,...,0,1,1,0,1,0,0,1,0,1


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


# Normalize the input features
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.values)

X = data.iloc[:, 2:].values
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train_tensor = torch.from_numpy(X_train).float()
X_test_tensor = torch.from_numpy(X_test).float()
y_train_tensor = torch.from_numpy(y_train).long()
y_test_tensor = torch.from_numpy(y_test).long()

# Create a TensorDataset and DataLoader for training data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

# Initialize the model
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
model_lstm = LSTMModel(input_size, hidden_size, output_size)

# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model_lstm.parameters(), lr=0.001)

# Train the model
num_epochs = 10
model_lstm.train()
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model_lstm(inputs.unsqueeze(dim=1))
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()

# Evaluate the model
model_lstm.eval()
with torch.no_grad():
    y_pred_tensor = torch.round(torch.sigmoid(model_lstm(X_test_tensor.unsqueeze(dim=1))))
    y_pred_lstm = y_pred_tensor.numpy()
    y_test_np = y_test_tensor.numpy()

accuracy = accuracy_score(y_test_np, y_pred_lstm)
print("Accuracy:", accuracy)

Accuracy: 0.8866082962537678


In [ ]:
import plotly.graph_objects as go

# Perform time series prediction
model_lstm.eval()
with torch.no_grad():
    # Use the trained model to predict on the entire dataset
    X_tensor = torch.from_numpy(X).float()
    y_pred_tensor = torch.round(torch.sigmoid(model_lstm(X_tensor.unsqueeze(dim=1))))
    y_pred_lstm = y_pred_tensor.numpy()

# Create the scatter plot
fig = go.Figure()

# Add the predicted values as scatter points
fig.add_trace(go.Scatter(
    x=data.index,
    y=target,
    mode="markers",
    name="Predicted",
    marker=dict(color="blue")
))


# Set the layout and display the figure
fig.update_layout(
    title="Time Series Prediction with LSTM",
    xaxis=dict(title="Date"),
    yaxis=dict(title="ГБ новая"),
    hovermode="x",
    showlegend=True
)
fig.show()


In [ ]:
# Evaluate the model and generate predictions for the test dataset
model_lstm.eval()
with torch.no_grad():
    y_pred_probs = torch.sigmoid(model_lstm(X_test_tensor.unsqueeze(dim=1)))
    y_pred_class = torch.round(y_pred_probs)

# Convert the tensors to numpy arrays
y_pred_probs_np = y_pred_probs.numpy()
y_pred_class_np = y_pred_class.numpy()

# Create a report with the predicted class value and probability for each sample
report = []
for i in range(len(X_test)):
    sample_report = {
        'Sample': i + 1,
        'Actual Class': y_test[i],
        'Predicted Class': y_pred_class_np[i],
        'Probability': 1 - y_pred_probs_np[i]
    }
    report.append(sample_report)

# Print the report
for sample_report in report:
    print(sample_report)


Streaming output truncated to the last 5000 lines.
{'Sample': 1968, 'Actual Class': 0, 'Predicted Class': array([0.], dtype=float32), 'Probability': array([0.94857645], dtype=float32)}
{'Sample': 1969, 'Actual Class': 0, 'Predicted Class': array([0.], dtype=float32), 'Probability': array([0.9538968], dtype=float32)}
{'Sample': 1970, 'Actual Class': 0, 'Predicted Class': array([0.], dtype=float32), 'Probability': array([0.8312176], dtype=float32)}
{'Sample': 1971, 'Actual Class': 0, 'Predicted Class': array([0.], dtype=float32), 'Probability': array([0.8662952], dtype=float32)}
{'Sample': 1972, 'Actual Class': 0, 'Predicted Class': array([0.], dtype=float32), 'Probability': array([0.95408064], dtype=float32)}
{'Sample': 1973, 'Actual Class': 0, 'Predicted Class': array([0.], dtype=float32), 'Probability': array([0.94554573], dtype=float32)}
{'Sample': 1974, 'Actual Class': 0, 'Predicted Class': array([0.], dtype=float32), 'Probability': array([0.89585], dtype=float32)}
{'Sample': 1975, 